In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import re
import nltk

%matplotlib inline

In [2]:
data = pd.read_csv('data.csv')

In [37]:
data.head()

,Category,Text
0,Farmland,US land prices extend decline - but at slower ...
1,Farmland,Dry weather forces BrasilAgro to slash corn so...
2,Farmland,Australia delays ruling on $290m China-backed ...
3,Farmland,Demand for high-end dairy farms in NZ shows 'l...
4,Farmland,FirstFarms unveils plan to exploit Slovakian l...


In [4]:
np.unique(data['Category'])

array(['Canola', 'Cocoa', 'Coffee', 'Corn', 'Cotton & wool', 'Dairy',
       'Farmland', 'Fertilizer', 'Hay & Forage', 'Horticulture',
       'Livestock & poultry', 'Pulse', 'Soybeans', 'Soybeans & oilseeds',
       'Sugar', 'Wheat', 'Wheat & cereals'], dtype=object)

In [36]:
data['Category'].value_counts()

Soybeans               5904
Wheat                  5563
Corn                   3875
Hay & Forage           2632
Canola                 2150
Pulse                  1644
Horticulture            703
Dairy                    13
Coffee                   13
Livestock & poultry      13
Farmland                 13
Fertilizer               13
Cocoa                    13
Cotton & wool            13
Sugar                    13
Name: Category, dtype: int64

In [14]:
def combine_categories(x):
    if x=='Soybeans & oilseeds':
        return 'Soybeans'
    elif x=='Wheat & cereals':
        return 'Wheat'
    else:
        return x

data['Category'] = data['Category'].apply(combine_categories)

In [19]:
count_data = data['Category'].value_counts()

In [20]:
count_data

Soybeans               5904
Wheat                  5563
Corn                   3875
Hay & Forage           2632
Canola                 2150
Pulse                  1644
Horticulture            703
Dairy                    13
Coffee                   13
Livestock & poultry      13
Farmland                 13
Fertilizer               13
Cocoa                    13
Cotton & wool            13
Sugar                    13
Name: Category, dtype: int64

In [44]:
leaveout = count_data[count_data<800]
list(leaveout.index)

['Horticulture',
 'Dairy',
 'Coffee',
 'Livestock & poultry',
 'Farmland',
 'Fertilizer',
 'Cocoa',
 'Cotton & wool',
 'Sugar']

In [45]:
sub_data = data
for col in list(leaveout.index):
    sub_data = sub_data[sub_data['Category'] != col]

In [46]:
sub_data['Category'].value_counts()

Soybeans        5904
Wheat           5563
Corn            3875
Hay & Forage    2632
Canola          2150
Pulse           1644
Name: Category, dtype: int64

In [12]:
def listofbadwords():
    from nltk.corpus import stopwords
    stopwords = stopwords.words('english')
    monthnames = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
    randomrepetitive = ['edu','unl']
    
    totlist = stopwords + monthnames + randomrepetitive
    return totlist

totlist = listofbadwords()
    

In [14]:
sub_data = pd.read_csv('sub_data.csv')

In [15]:
def replaceredundancy(x):
    retstr = x.replace('Click here to see more...', '')
    return retstr

import string
def removepunctuation(x):
    #x = x.replace('.',' ')
    #x = x.replace(')',' ')
    #x = x.replace('(',' ')
    replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))
    x = x.translate(replace_punctuation)
    #retstr = x.translate(string.maketrans("",""), string.punctuation)
    return x
    
def removeunicode(x):
    return re.sub(r'[^\x00-\x7F]+',' ', x)

def trimstring(x):
    x = x.lstrip()
    x= x.rstrip()
    return x

def lowercasestring(x):
    return x.lower()

def removedigits(s):
    s = re.sub(" \d+", " ", s)
    return s

def removebadwords(x):
    
    wordlist = x.split()
    wordlist = [word for word in wordlist if word.lower() not in totlist]
    x = ' '.join(wordlist)
    return x
    
def cleanstring(x):
    x=replaceredundancy(x)
    x=removepunctuation(x)
    x=removeunicode(x)
    x = trimstring(x)
    x=removedigits(x)
    x=lowercasestring(x)
    
    x = removebadwords(x)
    return x 

In [16]:
clean_data = sub_data.copy(deep=True)
clean_data['Text'] = clean_data['Text'].apply(cleanstring)

In [20]:
clean_data.to_excel('clean.xlsx')